# B站：北大TensorFlow笔记

## 目标1：
搭建神经网络，总结搭建的八股
### TensorFlow中的神经网络

#### 整个计算过程
* 张量表示数据
* 计算图搭建神经网络
* 会话执行计算图
* 优化线上权重（参数）
* 得到模型

In [2]:
#coding:utf-8
import tensorflow as tf
import numpy as np

#### 张量表示数据
1.先用`constant`构建两个常数向量的变量  
2.两个向量做加法，返回值

关于相应的名称
* 乘法：mul
* 除法：truediv
* 加法：add
* 减法：sub
* 取余数：mod

In [3]:
a = tf.constant([1.0, 2.0])
b = tf.constant([3.0, 4.0])
result = a % b
print(result)

Tensor("mod:0", shape=(2,), dtype=float32)


#### 计算图（Graph）搭建神经网路

**能print()出以下内容的变量都称为计算图，即有加减乘除内容的运算**  
`Tensor("mod:0", shape=(2,), dtype=float32)`

* 计算图参数的解读：  
    * 张量名称`add:0`
        * add->节点名  
        * 0->第0个输出  
    * shape->维度（2,）表示一维数组，长度为2  
    * dtype=float32->数据类型

**图只负责搭建网络，不进行计算**

**矩阵相乘案例**

* 一个中括号内有几个数，就代表有几列数
* 一个中括号内有几个中括号，就代表有几行数

In [15]:
x = tf.constant([[1.0, 2.0]])
w = tf.constant([[3.0], [4.0]])
y = tf.matmul(x, w)
print(y)

Tensor("MatMul_6:0", shape=(1, 1), dtype=float32)


#### 使用会话（Session）进行计算，执行计算图
使用with实现，结构如下
```
with tf.Session() as sess:
    print(sess.run(y))
```

In [5]:
x = tf.constant([[1.0, 2.0]])
w = tf.constant([[3.0], [4.0]])
y = tf.matmul(x, w)
print(y)

with tf.Session() as sess:
    print(sess.run(y))

Tensor("MatMul_1:0", shape=(1, 1), dtype=float32)
[[ 11.]]


#### 优化权重（参数）

**1.首先随机生成参数`tf.Variable()`**  
`w = tf.Variable(tf.random_normal([2,3], stddev=2, mean=0, seed=1))`

* tf.random_normal：生成正态分布的随机数
    * 大小[2,3]
    * 以下没有特殊要求可以不写：
        * stddev：标准差=2
        * mean：均值为0
        * seed：随机种子为1（表示做随机数的序列，固定随机种子，每次生成的随机数保持一致）

**2.关于生成随机数的分布，有以下几种：**  
* tf.random_normal()：正态分布
* tf.truncated_normal()：去掉过大偏离点的正态分布（如果随机出的数据超出平均值两个标准差，则会被去掉）
* tf.random_uniform()：平均分布

**3.生成数组常量**
* tf.zeros，生成全0数组，`tf.zeros([3,2],int32)`，生成三行两列全0矩阵
* tf.ones，生成全1数组，`tf.ones([3,2],int32)`，生成三行两列全1矩阵
* tf.fill，生成全定值数组，`tf.fill([3,2],6)`，生成三行两列全1矩阵
* tf.constant，根据给定的数值生成数组（前面用到过）

In [6]:
q = tf.zeros([3,2], tf.int32)
q = tf.ones([3,2], tf.int32)
q = tf.fill([3,2], 6)

with tf.Session() as sess:
    print(sess.run(q))

[[6 6]
 [6 6]
 [6 6]]


### 神经网络的实现

**整个过程：**

* 训练模型  
1.准备数据集，提取特征，作为输入喂给NN  
2.从输入到输出，搭建NN结构（前向传播）：先搭建计算图，再执行会话  
3.将特征数据喂给NN，迭代优化NN权值参数（反向传播） 


* 使用模型  
4.将训练好的模型进行预测和分类  

### 构建前向传播

#### 变量初始化
* 1.定义变量初值（搭建框架）
初始化所有变量的函数，简写为初始化节点`init_op`
```
x = tf.constant([[0.7, 0.5]])
w1 = tf.Variable(tf.random_normal([2, 3],stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1],stddev=1, seed=1))
```
* 2.执行初始化（赋予变量初值）
```
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
```

#### 张量乘法
* 1.定义张量之间的关系（搭建框架）
```
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)
```
* 2.执行乘法
```
with tf.Session() as sess:
    print("y is:\n", sess.run(y))
```


#### Other Points
1.参数矩阵的具体形状  
* 本层指的是计算层（W1是指第一个计算层，W2指的是第二层，因此不包含输入层）  
* (上层节点个数, 本层节点个数）

2.当前为什么没有用到激活函数和偏置值b？

In [14]:
# coding:utf-8
# 两层全连接神经网络demo

# 1.定义输入、随机正态分布初始化权重参数的初值
# 定义一个固定输入，使用tf.constant
x = tf.constant([[0.7, 0.5]])
w1 = tf.Variable(tf.random_normal([2, 3],stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1],stddev=1, seed=1))

# 2.定义前向传播过程中变量之间的关系（相乘）
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

# 3.使用Session计算结果
with tf.Session() as sess:
    # 01.初始化变量
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    # 02.进行NN计算，并且输出结果
    print("y is:\n", sess.run(y))

y is:
 [[ 3.0904665]]


### 构建反向传播

#### 喂数据
先用`tf.placeholder`进行占位，shape=(数据个数, 数据维度)，告诉`sess.run(y)`要输入多少个数据

* 喂一组数据，数据有两个特征：
先占一个位置的数据，然后输入数据
```
x = tf.placeholder(tf.float32,shape=(1, 2))
sess.run(y, feed_dict={x:[[0.5, 0.6]]})
```

* 喂多组数据，数据同样是有两维特征：
数据个数填写None，代表无数个数据/未知
```
x = tf.placeholder(tf.float32,shape=(None, 2))
sess.run(y, feed_dict={x:[[0.1,0.2], [0.2,0.3], [0.3,0.4], [0.4,0.5]]})
```

##### 向NN喂入一组特征

In [20]:
# coding:utf-8
# 两层全连接神经网络demo

# 1.定义输入、随机正态分布初始化权重参数的初值
# 定义一个输入，使用tf.placeholder()
x = tf.placeholder(tf.float32, shape=(1, 2))
w1 = tf.Variable(tf.random_normal([2, 3],stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1],stddev=1, seed=1))

# 2.定义前向传播过程中变量之间的关系（相乘）
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

# 3.使用Session计算结果
with tf.Session() as sess:
    # 01.初始化变量
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    # 02.使用feed_dict喂入参数，进行NN计算，并且输出结果
    print("y is:\n", sess.run(y, feed_dict={x: [[0.7, 0.5]]}))

y is:
 [[ 3.0904665]]


##### 向NN喂入多组特征

In [21]:
# coding:utf-8
# 两层全连接神经网络demo

# 1.定义输入、随机正态分布初始化权重参数的初值
# 定义多个输入，使用tf.placeholder()
x = tf.placeholder(tf.float32, shape=(None, 2))
w1 = tf.Variable(tf.random_normal([2, 3],stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1],stddev=1, seed=1))

# 2.定义前向传播过程中变量之间的关系（相乘）
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

# 3.使用Session计算结果
with tf.Session() as sess:
    # 01.初始化变量
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    # 02.使用feed_dict喂入参数，进行NN计算，并且输出结果
    print("y is:\n", sess.run(y, feed_dict={x: [[0.7, 0.5], [0.2, 0.3], [0.3, 0.4], [0.4, 0.5]]}))
    print("w1:\n", sess.run(w1))
    print("w2:\n", sess.run(w2))

y is:
 [[ 3.0904665 ]
 [ 1.2236414 ]
 [ 1.72707319]
 [ 2.23050475]]
w1:
 [[-0.81131822  1.48459876  0.06532937]
 [-2.4427042   0.0992484   0.59122431]]
w2:
 [[-0.81131822]
 [ 1.48459876]
 [ 0.06532937]]


#### 损失函数
常见的损失函数
* MSE均方误差
即为预测值与真实值差的平方，除以样本个数n  
`loss = tf.reduce_mean(tf.square(y_-y))`

#### 反向传播训练
目标：减小loss数值  
常见方法：
* 梯度下降  
`train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)`
* Momentum优化  
`train_step = tf.train.MomentumOptimizer(learining_rate, momentum).minimize(loss)`
* Adam  
`train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)`

其中，学习率learning_rate代表参数每次更新的幅度

**代码案例如下：需要背下来**

In [7]:
#coding:utf-8
import tensorflow as tf
import numpy as np

* 1.生成数据集

In [8]:
# batch_size：一次喂给神经网络多少数据（此数值不可以过大，否则会吃不消）
BATCH_SIZE = 8
seed = 23455

# 基于随机种子23455生成数据集
rng = np.random.RandomState(seed)
# 生成32行数据，每组数据都有 体积和重量 两个属性作为特征
X = rng.rand(32,2)
Y = [[int(x0 + x1 < 1)] for (x0, x1) in X]

print("X:\n",X)
print("Y:\n",Y)

* 2.搭建NN的输入、输出、参数
* 3.搭建NN的前向传播过程

In [ ]:
# 搭建NN的输入、输出、参数（其中y是矩阵计算后的值，而y_是从矩阵Y中取出来的标签
x = tf.placeholder(tf.float32, shape=(None, 2))
y_= tf.placeholder(tf.float32, shape=(None, 1))

w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

# 搭建NN的前向传播过程
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

* 4.反向传播，指定损失函数loss

In [57]:
# 均方误差计算损失
loss = tf.reduce_mean(tf.square(y-y_))

# 梯度下降开始学习，学习率0.001
# train_step = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

train_step = tf.train.MomentumOptimizer(0.001, 0.9).minimize(loss)
# train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

#### 生成Session，训练steps轮

In [58]:
with tf.Session() as sess:
    # 初始化变量
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    # 输出训练前的权重
    print("w1:\n", sess.run(w1))
    print("w2:\n", sess.run(w2))
    print("\n")
    
    # 训练模型
    STEPS = 12001
    for i in range(STEPS):
        start = (i*BATCH_SIZE) % 32
        end = start + BATCH_SIZE
        # 分批喂入训练数据，进行权重的学习（梯度下降法）
        sess.run(train_step, feed_dict={x:X[start:end], y_:Y[start:end]})
#         print("train_step\n",train_step)
        
        # 输出训练后的权值参数
#         print("\n")
#         print("w1:\n",sess.run(w1))
#         print("w2:\n",sess.run(w2))
        
        # 每500次计算一次均方误差
        if i % 500 == 0:
            total_loss = sess.run(loss, feed_dict={x:X, y_:Y})
            print("After %d training steps, loss_mse on all data is %g" % (i+1, total_loss))

w1:
 [[-0.81131822  1.48459876  0.06532937]
 [-2.4427042   0.0992484   0.59122431]]
w2:
 [[-0.81131822]
 [ 1.48459876]
 [ 0.06532937]]


After 1 training steps, loss_mse on all data is 5.13118
After 501 training steps, loss_mse on all data is 0.384391
After 1001 training steps, loss_mse on all data is 0.383592
After 1501 training steps, loss_mse on all data is 0.383562
After 2001 training steps, loss_mse on all data is 0.383561
After 2501 training steps, loss_mse on all data is 0.383561
After 3001 training steps, loss_mse on all data is 0.383561
After 3501 training steps, loss_mse on all data is 0.383561
After 4001 training steps, loss_mse on all data is 0.383561
After 4501 training steps, loss_mse on all data is 0.383561
After 5001 training steps, loss_mse on all data is 0.383561
After 5501 training steps, loss_mse on all data is 0.383561
After 6001 training steps, loss_mse on all data is 0.383561
After 6501 training steps, loss_mse on all data is 0.383561
After 7001 training steps, l

## 目标2：优化神经网络

### 损失函数 loss
* 目标：使得损失loss最小
* 常见方法：
    * 均方误差MSE(Mean Squared Error)  
    `loss_mse = tf.reduce_mean(tf.square(y_-y))`
    * 自定义
    * 交叉熵CE(Cross Entropy)
    
#### 均方误差MSE案例
例：建模牛奶的销量，以预测牛奶的产量  
本案例只有一个输出层，即单层神经网络  
本案例中，默认预测多了与预测少了损失相同。（但是实际情况，预测多了损失成本，预测少了损失利润）  
预期生成的应该是X1+X2，实际生成的是0.98X1+1.02X2，说明预测酸奶日销量的函数拟合正确

In [12]:
#coding:utf-8
# 导入模块
import tensorflow as tf
import numpy as np

# 生成数据集
BATCH_SIZE = 8
SEED = 23455
rdm = np.random.RandomState(SEED)
X = rdm.rand(32,2)
# 这里虽数据加入了噪声: -0.05 ~ +0.05
Y_ = [[x1+x2+(rdm.rand()/10.0-0.05)] for (x1, x2) in X]


# 定义神经网络输入、输出、权重，定义前向传播过程
x = tf.placeholder(tf.float32, shape=(None, 2))
y_ = tf.placeholder(tf.float32, shape=(None, 1))
w1 = tf.Variable(tf.random_normal([2,1], stddev=1, seed=1))
y = tf.matmul(x,w1)

# 定义损失函数MSE & 反向传播（方法为梯度下降法）
loss_mse = tf.reduce_mean(tf.square(y_-y))
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(loss_mse)
# train_step = tf.train.MomentumOptimizer(0.001, 0.9).minimize(loss_mse)
# train_step = tf.train.AdamOptimizer(0.001).minimize(loss_mse)

# 生成Session，启动训练，进行STEPS轮
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    STEPS = 20000
    for i in range(STEPS):
        start = (i*BATCH_SIZE) % 32
        end = start + BATCH_SIZE
        sess.run(train_step, feed_dict={x: X[start:end], y_:Y_[start:end]})
        if i % 500 == 0:
            print("After %d training steps, w1 is: " % (i+1))
            print(sess.run(w1),"\n")
    print("Final w1 is: \n", sess.run(w1))

After 1 training steps, w1 is: 
[[-0.80974597]
 [ 1.48529029]] 

After 501 training steps, w1 is: 
[[-0.46074435]
 [ 1.64187801]] 

After 1001 training steps, w1 is: 
[[-0.21939856]
 [ 1.69847655]] 

After 1501 training steps, w1 is: 
[[-0.04415595]
 [ 1.70031762]] 

After 2001 training steps, w1 is: 
[[ 0.08942621]
 [ 1.67332804]] 

After 2501 training steps, w1 is: 
[[ 0.19583553]
 [ 1.63226771]] 

After 3001 training steps, w1 is: 
[[ 0.28375748]
 [ 1.58544338]] 

After 3501 training steps, w1 is: 
[[ 0.35848638]
 [ 1.53744709]] 

After 4001 training steps, w1 is: 
[[ 0.42332521]
 [ 1.49073923]] 

After 4501 training steps, w1 is: 
[[ 0.48040032]
 [ 1.44655728]] 

After 5001 training steps, w1 is: 
[[ 0.5311361 ]
 [ 1.40545344]] 

After 5501 training steps, w1 is: 
[[ 0.57653254]
 [ 1.367594  ]] 

After 6001 training steps, w1 is: 
[[ 0.6173259 ]
 [ 1.33294022]] 

After 6501 training steps, w1 is: 
[[ 0.65408474]
 [ 1.30134249]] 

After 7001 training steps, w1 is: 
[[ 0.68726856]
 [

#### 自定义损失案例

考虑成本和损失不相同，使用**自定义损失函数**来计算

制定分段函数：
预测多了，损失成本；预测少了，损失利润（单件成本1元，利润9元），因此生产越多越好（如果利润是1，成本是9，那么NN会朝着尽量少生产的方向走）  
使用`greater()`函数判断两个变量的大小，最后将损失求和  
`loss = tf.reduce_sum(tf.where(tf.greater(y,y_)，COST(y-y_), PROFIT(y_-y)))`  

In [14]:
#coding:utf-8
# 导入模块
import tensorflow as tf
import numpy as np

# 生成数据集
BATCH_SIZE = 8
SEED = 23455
COST = 1
PROFIT = 9

rdm = np.random.RandomState(SEED)
X = rdm.rand(32,2)
# 这里虽数据加入了噪声: -0.05 ~ +0.05
Y_ = [[x1+x2+(rdm.rand()/10.0-0.05)] for (x1, x2) in X]


# 定义神经网络输入、输出、权重，定义前向传播过程
x = tf.placeholder(tf.float32, shape=(None, 2))
y_ = tf.placeholder(tf.float32, shape=(None, 1))
w1 = tf.Variable(tf.random_normal([2,1], stddev=1, seed=1))
y = tf.matmul(x,w1)

# 自定义损失函数 & 反向传播（方法为梯度下降法）
loss_mse = tf.reduce_sum(tf.where(tf.greater(y, y_), (y - y_)*COST, (y_ - y)*PROFIT))
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(loss_mse)
# train_step = tf.train.MomentumOptimizer(0.001, 0.9).minimize(loss_mse)
# train_step = tf.train.AdamOptimizer(0.001).minimize(loss_mse)

# 生成Session，启动训练，进行STEPS轮
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    STEPS = 20000
    for i in range(STEPS):
        start = (i*BATCH_SIZE) % 32
        end = start + BATCH_SIZE
        sess.run(train_step, feed_dict={x: X[start:end], y_:Y_[start:end]})
        if i % 500 == 0:
            print("After %d training steps, w1 is: " % (i+1))
            print(sess.run(w1),"\n")
    print("Final w1 is: \n", sess.run(w1))

After 1 training steps, w1 is: 
[[-0.80594873]
 [ 1.48737288]] 

After 501 training steps, w1 is: 
[[ 0.8732146 ]
 [ 1.00620401]] 

After 1001 training steps, w1 is: 
[[ 0.96580642]
 [ 0.96982086]] 

After 1501 training steps, w1 is: 
[[ 0.96454471]
 [ 0.96829468]] 

After 2001 training steps, w1 is: 
[[ 0.96024752]
 [ 0.97420847]] 

After 2501 training steps, w1 is: 
[[ 0.96100295]
 [ 0.96993423]] 

After 3001 training steps, w1 is: 
[[ 0.96541017]
 [ 0.97611594]] 

After 3501 training steps, w1 is: 
[[ 0.96414846]
 [ 0.97458977]] 

After 4001 training steps, w1 is: 
[[ 0.95985126]
 [ 0.98050356]] 

After 4501 training steps, w1 is: 
[[ 0.96364218]
 [ 0.96878934]] 

After 5001 training steps, w1 is: 
[[ 0.95934498]
 [ 0.97470313]] 

After 5501 training steps, w1 is: 
[[ 0.9667877 ]
 [ 0.97344488]] 

After 6001 training steps, w1 is: 
[[ 0.96419948]
 [ 0.96766263]] 

After 6501 training steps, w1 is: 
[[ 0.96557122]
 [ 0.98128432]] 

After 7001 training steps, w1 is: 
[[ 0.96530831]
 [

#### 交叉熵CE案例
交叉熵：表征两个概率分布的距离（交叉熵越大，两个概率分布越远，交叉熵越小，两个概率分布越近）

工程中不会用到以下函数：
`ce = -tf.reduce_mean(y_*tf.log(tf.clip_by_value(y, 1e-12, 1.0)))`

![](https://raw.githubusercontent.com/king0682/FigureBed/master/img/20190718225630.png)

**softmax函数目的：**  
1.为了让前向传播的结果y满足概率分布，让n分类的n个输出，每个都在[0,1]之间  
2.且n个输出之和为1  

![](https://raw.githubusercontent.com/king0682/FigureBed/master/img/20190718230434.png)

**实际操作：**

1.输出经过softmax函数，使其满足概率分布  
2.将输出结果与标准答案求交叉熵  

工程中用到以下两句函数(cem是损失函数，在后面手写数字识别会用)：
```
ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
cem = tf.reduce_mean(ce)
```

### 学习率 learning rate
![](https://raw.githubusercontent.com/king0682/FigureBed/master/img/20190718231738.png)

**案例：**  
损失函数loss = (w+1)^2, w初值为5，反向传播找最优w，即对应loss最小的w值

In [15]:
#coding:utf-8
import tensorflow as tf

# 定义待优化参数w初值为5
w = tf.Variable(tf.constant(5, dtype = tf.float32))

# 定义损失函数loss
loss = tf.square(w+1)

# 定义反向传播
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

# 生成Session，进行40轮训练
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    for i in range(40):
        sess.run(train_step)
        w_val = sess.run(w)
        loss_val = sess.run(loss)
        print(("After %s steps: w is %f, loss is %f.") % (i, w_val, loss_val))

After 0 steps: w is 2.600000, loss is 12.959999.
After 1 steps: w is 1.160000, loss is 4.665599.
After 2 steps: w is 0.296000, loss is 1.679616.
After 3 steps: w is -0.222400, loss is 0.604662.
After 4 steps: w is -0.533440, loss is 0.217678.
After 5 steps: w is -0.720064, loss is 0.078364.
After 6 steps: w is -0.832038, loss is 0.028211.
After 7 steps: w is -0.899223, loss is 0.010156.
After 8 steps: w is -0.939534, loss is 0.003656.
After 9 steps: w is -0.963720, loss is 0.001316.
After 10 steps: w is -0.978232, loss is 0.000474.
After 11 steps: w is -0.986939, loss is 0.000171.
After 12 steps: w is -0.992164, loss is 0.000061.
After 13 steps: w is -0.995298, loss is 0.000022.
After 14 steps: w is -0.997179, loss is 0.000008.
After 15 steps: w is -0.998307, loss is 0.000003.
After 16 steps: w is -0.998984, loss is 0.000001.
After 17 steps: w is -0.999391, loss is 0.000000.
After 18 steps: w is -0.999634, loss is 0.000000.
After 19 steps: w is -0.999781, loss is 0.000000.
After 20 ste

学习率设置多少合适？  
使用指数衰减学习率，可以根据BATCH_SIZE的轮数动态更新学习率  
![](https://raw.githubusercontent.com/king0682/FigureBed/master/img/20190718233645.png)

In [20]:
#coding:utf-8
import tensorflow as tf

LEARNING_RATE_BASE = 0.1 # 最初学习率
LEARNING_RATE_DECAY = 0.99 # 学习率衰减率
LEARNING_RATE_STEP = 1 # 指喂入多少轮BATCH_SIZE后，更新一次学习率，一般设置为：总样本数/BATCH_SIZE

# 定义BATCH_SIZE训练轮数计数器，初值为0，该变量设置为不可训练
global_step = tf.Variable(0, trainable = False)

# 定义指数下降学习率
learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, global_step, LEARNING_RATE_STEP, LEARNING_RATE_DECAY, staircase=True)

# 定义待优化参数w。初值为10
w = tf.Variable(tf.constant(10, dtype = tf.float32))

# 定义损失函数
loss = tf.square(w+1)

# 定义反向传播
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

# 生成Session，训练40轮
with  tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    for i in range(40):
        sess.run(train_step)
        learning_rate_val = sess.run(learning_rate)
        global_step_val = sess.run(global_step)
        w_val = sess.run(w)
        loss_val = sess.run(loss)
        print("After %s steps: global_step is %f, learning rate is %f, loss is %f" % (i, global_step_val, learning_rate_val, loss_val))

After 0 steps: global_step is 1.000000, learning rate is 0.099000, loss is 77.440002
After 1 steps: global_step is 2.000000, learning rate is 0.098010, loss is 49.809719
After 2 steps: global_step is 3.000000, learning rate is 0.097030, loss is 32.196201
After 3 steps: global_step is 4.000000, learning rate is 0.096060, loss is 20.912704
After 4 steps: global_step is 5.000000, learning rate is 0.095099, loss is 13.649124
After 5 steps: global_step is 6.000000, learning rate is 0.094148, loss is 8.950812
After 6 steps: global_step is 7.000000, learning rate is 0.093207, loss is 5.897362
After 7 steps: global_step is 8.000000, learning rate is 0.092274, loss is 3.903603
After 8 steps: global_step is 9.000000, learning rate is 0.091352, loss is 2.595742
After 9 steps: global_step is 10.000000, learning rate is 0.090438, loss is 1.733887
After 10 steps: global_step is 11.000000, learning rate is 0.089534, loss is 1.163375
After 11 steps: global_step is 12.000000, learning rate is 0.088638,

### 滑动平均 ema

### 正则化 regularization